In [1]:
import numpy as np 
import pandas as pd
from datetime import timedelta
import datetime as dt
import matplotlib.pyplot as plt
import pickle as cPickle
from os.path import exists
from pathlib import Path
import csv
import datetime
import pytz
import plotly.express as px
import os

from pathlib import Path 
import time

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
test = pd.read_csv('/content/drive/MyDrive/TNC_Collab_Results/testing_data_all_20220523.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491 entries, 0 to 490
Columns: 299 entries, Unnamed: 0 to wb2sum
dtypes: float64(290), int64(4), object(5)
memory usage: 1.1+ MB


In [3]:
usgs_reg = pd.read_csv('/content/drive/MyDrive/TNC_Collab_Results/USGS_region_lookup.csv')
usgs_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   NA_L3CODE   11 non-null     object
 1   USGS_Group  11 non-null     object
dtypes: object(2)
memory usage: 304.0+ bytes


In [4]:
region_dict = dict(zip(usgs_reg.NA_L3CODE, usgs_reg.USGS_Group))
test['NewEco3'] = test['NewEco3'].replace(region_dict)
print(test['NewEco3'].unique())

['Xeric' 'Interior Mountains' 'North Coastal Mountains']


In [5]:
test = test.loc[(test['NewEco3'] == 'North Coastal Mountains')] 
test

,Unnamed: 0,year,month,flow_cfs,comid,statistic,variable,ECO3,NHDPLUSREG,DRAIN_SQKM,...,wb9,wb10,wb11,wb12,p6sum,p3sum,p2sum,wb6sum,wb3sum,wb2sum
32,32,2021,11,6.047000,17684378,mean,observed,1.0,18.0,106.7481,...,40.933581,80.158666,3.150605,3.220339,173.204614,171.699478,171.450850,27.239413,10.689135,9.499916
33,33,2021,11,31.123333,17682474,mean,observed,1.0,18.0,274.9149,...,45.707054,82.878789,3.487040,3.019215,212.107778,210.302989,210.302165,45.212220,26.899377,25.597721
34,34,2021,11,5.232000,17688105,mean,observed,1.0,18.0,119.2950,...,25.109039,31.770949,3.394544,2.005575,197.630471,193.250510,193.250510,24.821847,16.521906,15.944696
35,35,2021,11,3.246000,17687965,mean,observed,1.0,18.0,133.8120,...,13.029880,11.604779,3.137244,1.587258,178.599392,172.692088,172.656603,14.614667,9.266237,8.903530
56,56,2021,11,253.900000,2495296,mean,observed,78.0,18.0,592.3971,...,79.381388,77.127615,5.475133,3.413667,258.121980,254.212700,254.212700,88.496307,59.923265,57.896296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,73,2022,4,1326.033333,8246426,mean,observed,78.0,18.0,1862.1864,...,7.601103,15.308777,30.186247,57.747175,679.476760,75.786819,26.752377,449.031760,173.426196,75.559734
487,74,2022,4,21.830000,8246406,mean,observed,78.0,18.0,58.2921,...,3.994904,8.205917,15.880099,32.167682,615.192896,74.808815,25.566962,393.744887,155.988664,66.718790
488,75,2022,4,8.854667,8245912,mean,observed,78.0,18.0,87.3180,...,3.512946,7.325463,14.036709,28.501177,593.585262,70.370874,24.678846,382.935572,144.507973,61.784059
489,76,2022,4,942.833333,8232392,mean,observed,78.0,18.0,1982.6784,...,5.655774,11.695629,22.618803,45.965040,678.279832,110.033255,54.818963,430.733201,172.723439,79.939388


In [6]:
#right now we have Nov 2021 - Apr 2022 data
test['month'].unique()

array([11, 12,  1,  2,  3,  4])

In [7]:
#no null columns
test = test.dropna(axis=0,how='any')
test.isnull().sum().sum()

0

In [8]:
test['flow_cfs'] = test['flow_cfs'] / test['DRAIN_SQKM']

In [9]:
#dropping inessential columns
col = ['comid','Unnamed: 0','year','statistic','variable','NHDPLUSREG','comidyear','ECO3','AggEcoRegion','NewEco3'] 
df = test.drop(columns = col)

#no need label encode as only have one region xeric
#label encoding
# cols_encoding = ['NewEco3']
# labelencoder = LabelEncoder()
# df['NewEco3_label'] = labelencoder.fit_transform(df['NewEco3'])
# df = df.drop(columns=cols_encoding)
# df.reset_index(drop=True, inplace=True)

In [10]:
def x_test(month):  
  test1 = df[df['month']==month]
  x_test = test1.drop(columns = ['flow_cfs']).copy()
  return x_test

def y_test(month):
  test1 = df[df['month']==month]
  y_test = test1.flow_cfs.values
  return y_test

November 2021

In [11]:
pickled_model_usgs_11 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Coastal_M-all-comids)/USGS_model_reduced_11_r2_0.921_2022-09-19 12:23:26.056244-07:00.pkl', 'rb'))
print(pickled_model_usgs_11)

RandomForestRegressor(max_depth=5, max_features=17, n_estimators=1000,
                      n_jobs=-1, verbose=1)


In [12]:
rf_var = pickled_model_usgs_11.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_usgs_11
predictions = rf.predict(x_test(11)) 
r2_test = r2_score(y_test(11)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.2s finished


Reduced r-squared scores: 0.73


December 2021

In [13]:
pickled_model_usgs_12 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Coastal_M-all-comids)/USGS_model_reduced_12_r2_0.9286_2022-09-19 12:28:19.123934-07:00.pkl', 'rb'))
print(pickled_model_usgs_12)

RandomForestRegressor(max_depth=5, max_features=17, n_estimators=1000,
                      n_jobs=-1, verbose=1)


In [14]:
rf_var = pickled_model_usgs_12.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_usgs_12
predictions = rf.predict(x_test(12)) 
r2_test = r2_score(y_test(12)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.2s finished


Reduced r-squared scores: 0.36


January 2022

In [15]:
pickled_model_usgs_1 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Coastal_M-all-comids)/USGS_model_reduced_1_r2_0.8964_2022-09-19 12:29:05.736725-07:00.pkl', 'rb'))
print(pickled_model_usgs_1)

RandomForestRegressor(max_depth=5, max_features=17, n_estimators=1000,
                      n_jobs=-1, verbose=1)


In [16]:
rf_var = pickled_model_usgs_1.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_usgs_1
predictions = rf.predict(x_test(1)) 
r2_test = r2_score(y_test(1)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.2s finished


Reduced r-squared scores: 0.47


February 2022

In [17]:
pickled_model_usgs_2 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Coastal_M-all-comids)/USGS_model_reduced_2_r2_0.8907_2022-09-19 12:29:52.707119-07:00.pkl', 'rb'))
print(pickled_model_usgs_2)

RandomForestRegressor(max_depth=5, max_features=17, n_estimators=1000,
                      n_jobs=-1, verbose=1)


In [18]:
rf_var = pickled_model_usgs_2.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_usgs_2
predictions = rf.predict(x_test(2)) 
r2_test = r2_score(y_test(2)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.2s finished


Reduced r-squared scores: -8.71


March 2022

In [19]:
pickled_model_usgs_3 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Coastal_M-all-comids)/USGS_model_reduced_3_r2_0.8847_2022-09-19 12:30:42.492001-07:00.pkl', 'rb'))
print(pickled_model_usgs_3)

RandomForestRegressor(max_depth=5, max_features=17, n_estimators=1000,
                      n_jobs=-1, verbose=1)


In [20]:
rf_var = pickled_model_usgs_3.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_usgs_3
predictions = rf.predict(x_test(3)) 
r2_test = r2_score(y_test(3)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.2s finished


Reduced r-squared scores: -0.95


April 2022

In [21]:
pickled_model_usgs_4 = cPickle.load(open('/content/drive/MyDrive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Coastal_M-all-comids)/USGS_model_reduced_4_r2_0.8501_2022-09-19 12:31:31.595567-07:00.pkl', 'rb'))
print(pickled_model_usgs_4)

RandomForestRegressor(max_depth=5, max_features=17, n_estimators=1000,
                      n_jobs=-1, verbose=1)


In [22]:
rf_var = pickled_model_usgs_4.feature_names_in_
reduced_col = ['flow_cfs'] + str(rf_var).replace("'", "").replace("\n", "").replace("[", "").replace("]", "").split(" ")
df = test.loc[:, reduced_col]
rf = pickled_model_usgs_4
predictions = rf.predict(x_test(4)) 
r2_test = r2_score(y_test(4)+1, predictions+1) 
print('Reduced r-squared scores:', round(r2_test, 2))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.2s finished


Reduced r-squared scores: 0.48


May 2022